# 合成控制法 - 构建「虚拟」对照组

## 课前思考

**场景一：新城市上线**

你的公司准备在上海开展业务。上线后，上海的 GMV 增长了 30%。

老板问："这 30% 是我们业务带来的，还是上海经济本身就在增长?"

你想用 DID，但问题是：
- ❌ 只有上海一个城市接受处理
- ❌ 找不到完美的对照城市（北京太大，杭州太小）
- ❌ 平行趋势假设很难满足

**如何解决？**

💡 **核心思想：如果找不到一个完美的对照，那就用多个城市的「加权平均」来合成一个！**

---

## 学习目标

通过本节课，你将学会：

1. ✅ 理解合成控制的核心思想（为什么要合成）
2. ✅ 掌握权重估计方法（如何合成）
3. ✅ 学习推断方法（如何检验显著性）
4. ✅ 理解与 DID 的关系（什么时候用哪个）

---

In [ ]:
# 环境配置
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 颜色方案
COLORS = {
    'treated': '#EB5757',      # 红色 - 处理组
    'synthetic': '#2D9CDB',    # 蓝色 - 合成控制
    'donors': '#95A5A6',       # 灰色 - 供体池
    'gap': '#27AE60',          # 绿色 - 效应
    'placebo': '#F2994A'       # 橙色 - 安慰剂
}

print("环境配置完成")

---

# Part 1: 为什么需要合成控制

## 1.1 传统方法的困境

### 场景：评估加州禁烟法案的效果

- **1988 年**：加州实施 Proposition 99（烟草税 + 禁烟）
- **问题**：如何评估政策效果？

### 尝试 1：前后对比

```
政策前：100 包/人
政策后：80 包/人
结论：政策使吸烟量下降 20%
```

❌ **问题**：全国吸烟率都在下降（健康意识提升），这 20% 不全是政策效果！

### 尝试 2：DID（找一个对照州）

选择德州作为对照：

```
         1980  1985  1988  1990  1995
加州     120   110   100    80    70
德州     100    95    90    85    80
```

❌ **问题**：
1. 德州和加州差异太大（经济、文化、气候）
2. 平行趋势假设不满足（1980-1988 趋势不同）
3. 只用一个州，结果不稳健

### 尝试 3：多个对照州的平均

用其他 39 个州的平均值作为对照：

❌ **问题**：阿拉斯加和纽约完全不同，简单平均不合理！

---

## 1.2 合成控制的核心思想

💡 **关键洞察：不是找一个完美的对照，而是用多个不完美的对照"合成"一个完美的！**

### 形象比喻：调色

```
目标：    红色 + 绿色 + 蓝色 = 紫色
         40%    10%    50%

合成控制： 纽约 + 德州 + 佛州 = "虚拟加州"
          30%    20%    50%
```

### 数学表达

设：
- 加州（处理单位）：$Y_1^N$ (没有政策时的潜在结果)
- 其他州（供体池）：$Y_2, Y_3, ..., Y_{J+1}$

目标：找到权重 $W = (w_2, w_3, ..., w_{J+1})$，使得：

$$
\text{合成加州} = \sum_{j=2}^{J+1} w_j \cdot Y_j \approx Y_1^N
$$

约束：
- $w_j \geq 0$ （非负）
- $\sum w_j = 1$ （权重和为 1）

---

In [ ]:
# 生成示例数据：加州禁烟案例
np.random.seed(42)

# 时间跨度：1970-2000，1988 年是处理时点
years = np.arange(1970, 2001)
T0 = 1988  # 处理时点
pre_period = years < T0
post_period = years >= T0

# 加州：处理单位
california_trend = -2.0  # 下降趋势
california_base = 120
california_pre = california_base + california_trend * (years[pre_period] - 1970) + np.random.normal(0, 2, pre_period.sum())

# 政策效应：-15 包/人
treatment_effect = -15
california_post = california_base + california_trend * (years[post_period] - 1970) + treatment_effect + np.random.normal(0, 2, post_period.sum())
california = np.concatenate([california_pre, california_post])

# 供体池：其他州
states_data = {}
state_names = ['纽约', '德州', '佛州', '伊利诺伊', '宾州', '俄亥俄']

for i, state in enumerate(state_names):
    # 每个州有不同的基线和趋势
    base = np.random.uniform(90, 130)
    trend = np.random.uniform(-2.5, -1.5)
    noise = np.random.normal(0, 3, len(years))
    states_data[state] = base + trend * (years - 1970) + noise

# 创建 DataFrame
df = pd.DataFrame({
    'year': years,
    'california': california,
    **states_data
})

print("数据生成完成")
print(f"时间跨度: {years[0]} - {years[-1]}")
print(f"处理时点: {T0}")
print(f"供体池州数: {len(state_names)}")
df.head()

In [ ]:
# 可视化：为什么简单平均不够好
fig = go.Figure()

# 加州（处理单位）
fig.add_trace(go.Scatter(
    x=df['year'],
    y=df['california'],
    name='加州（处理单位）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'
))

# 其他州（供体池）
for state in state_names:
    fig.add_trace(go.Scatter(
        x=df['year'],
        y=df[state],
        name=state,
        line=dict(color=COLORS['donors'], width=1, dash='dot'),
        opacity=0.5
    ))

# 简单平均
simple_avg = df[state_names].mean(axis=1)
fig.add_trace(go.Scatter(
    x=df['year'],
    y=simple_avg,
    name='简单平均（等权重）',
    line=dict(color='orange', width=2, dash='dash'),
))

# 处理时点
fig.add_vline(x=T0, line_dash="dash", line_color="gray", 
              annotation_text="政策实施", annotation_position="top")

fig.update_layout(
    title='为什么简单平均不够好？',
    xaxis_title='年份',
    yaxis_title='香烟消费量（包/人/年）',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n观察：")
print("  1. 简单平均（橙色虚线）在政策前就和加州（红色）差距很大")
print("  2. 这说明等权重平均不能很好地模拟加州的趋势")
print("  3. 我们需要找到更好的权重！")

---

# Part 2: 方法原理

## 2.1 合成控制的数学框架

### 符号定义

- **单位**：$i = 1, 2, ..., J+1$
  - $i=1$：处理单位（如加州）
  - $i=2, ..., J+1$：供体池（其他州）

- **时间**：$t = 1, 2, ..., T$
  - $t = 1, ..., T_0$：前处理期
  - $t = T_0+1, ..., T$：后处理期

- **结果变量**：$Y_{it}$
  - $Y_{1t}^I$：处理单位在处理后的观测值
  - $Y_{1t}^N$：处理单位在没有处理时的潜在结果（反事实）

### 因果效应

我们想估计的是：

$$
\tau_t = Y_{1t}^I - Y_{1t}^N, \quad t > T_0
$$

**问题**：$Y_{1t}^N$ 不可观测！

**解决**：用供体池的加权组合来估计 $Y_{1t}^N$：

$$
\hat{Y}_{1t}^N = \sum_{j=2}^{J+1} w_j^* Y_{jt}
$$

---

## 2.2 权重的估计

### 核心思想

在 **前处理期**，找到权重 $W = (w_2, ..., w_{J+1})$，使得合成控制尽可能接近处理单位。

### 方法 1：结果变量匹配

最直接的想法：在前处理期，最小化处理单位和合成控制的差异：

$$
W^* = \arg\min_W \sum_{t=1}^{T_0} \left( Y_{1t} - \sum_{j=2}^{J+1} w_j Y_{jt} \right)^2
$$

约束：
- $w_j \geq 0$
- $\sum_{j=2}^{J+1} w_j = 1$

### 方法 2：协变量匹配（更常用）

不仅匹配结果变量，还匹配影响结果的协变量（$X$）：

定义特征向量：

$$
X_1 = \begin{pmatrix}
\text{1980年吸烟率} \\
\text{人均收入} \\
\text{受教育程度} \\
\text{啤酒消费} \\
... \\
\end{pmatrix}, \quad
X_j = \begin{pmatrix}
\text{州 j 的对应特征}
\end{pmatrix}
$$

优化目标：

$$
W^* = \arg\min_W \left\| X_1 - \sum_{j=2}^{J+1} w_j X_j \right\|_V
$$

其中 $\|\cdot\|_V$ 是加权范数，$V$ 是对角权重矩阵，体现不同特征的重要性。

---

## 2.3 优化问题的求解

这是一个 **二次规划问题**（Quadratic Programming）：

$$
\begin{aligned}
\min_W \quad & (X_1 - X_0 W)^T V (X_1 - X_0 W) \\
\text{s.t.} \quad & w_j \geq 0, \quad \forall j \\
& \sum_{j=2}^{J+1} w_j = 1
\end{aligned}
$$

其中：
- $X_0 = [X_2, X_3, ..., X_{J+1}]$：供体池特征矩阵
- $V$：权重矩阵（通常通过交叉验证选择）

可以用 `scipy.optimize.minimize` 求解。

---

## 2.4 代码实现：合成控制估计器

下面我们实现一个完整的合成控制估计器。

In [ ]:
class SyntheticControl:
    """
    合成控制估计器
    
    参数:
    ------
    treatment_period : int
        处理开始的时间索引
    """
    
    def __init__(self, treatment_period):
        self.treatment_period = treatment_period
        self.weights = None
        self.synthetic_control = None
        self.treatment_effect = None
        
    def fit(self, treated, donors, method='outcome'):
        """
        估计合成控制权重
        
        参数:
        ------
        treated : array-like, shape (T,)
            处理单位的时间序列
        donors : array-like, shape (T, J)
            供体池的时间序列矩阵
        method : str
            'outcome': 只匹配结果变量
            'covariate': 匹配协变量（未实现）
        """
        treated = np.array(treated)
        donors = np.array(donors)
        
        # 提取前处理期数据
        treated_pre = treated[:self.treatment_period]
        donors_pre = donors[:self.treatment_period, :]
        
        # 优化目标：最小化前处理期的 RMSE
        def objective(w):
            synthetic = donors_pre @ w
            return np.sum((treated_pre - synthetic) ** 2)
        
        # 约束条件
        constraints = (
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # 权重和为 1
        )
        
        # 边界：权重非负
        bounds = [(0, 1) for _ in range(donors.shape[1])]
        
        # 初始值：等权重
        w0 = np.ones(donors.shape[1]) / donors.shape[1]
        
        # 求解
        result = minimize(
            objective,
            w0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'ftol': 1e-9, 'maxiter': 1000}
        )
        
        if not result.success:
            print("优化未收敛")
        
        self.weights = result.x
        
        # 生成合成控制
        self.synthetic_control = donors @ self.weights
        
        # 计算处理效应
        self.treatment_effect = treated[self.treatment_period:] - self.synthetic_control[self.treatment_period:]
        
        return self
    
    def get_weights(self):
        """返回权重"""
        return self.weights
    
    def get_effect(self):
        """返回平均处理效应"""
        return np.mean(self.treatment_effect)
    
    def predict(self):
        """返回合成控制的预测值"""
        return self.synthetic_control

print("合成控制估计器实现完成")

In [ ]:
# 应用合成控制
T0_index = np.where(df['year'] == T0)[0][0]  # 处理时点的索引

# 准备数据
treated = df['california'].values
donors = df[state_names].values

# 拟合模型
sc = SyntheticControl(treatment_period=T0_index)
sc.fit(treated, donors)

# 获取权重
weights = sc.get_weights()
weight_df = pd.DataFrame({
    '州': state_names,
    '权重': weights
}).sort_values('权重', ascending=False)

print("\n合成控制权重：")
print(weight_df)
print(f"\n平均处理效应: {sc.get_effect():.2f} 包/人/年")

In [ ]:
# 可视化：权重分布
fig = go.Figure()

fig.add_trace(go.Bar(
    x=weight_df['州'],
    y=weight_df['权重'],
    marker_color=COLORS['synthetic'],
    text=weight_df['权重'].round(3),
    textposition='auto',
))

fig.update_layout(
    title='合成控制权重分布',
    xaxis_title='州',
    yaxis_title='权重',
    template='plotly_white',
    height=400
)

fig.show()

print("\n解读：")
print(f"  • 权重最大的州: {weight_df.iloc[0]['州']} ({weight_df.iloc[0]['权重']:.1%})")
print(f"  • 权重为 0 的州: {(weight_df['权重'] < 0.01).sum()} 个")
print("  • 这说明只有少数几个州对合成加州贡献最大")

In [ ]:
# 可视化：处理单位 vs 合成控制
synthetic = sc.predict()

fig = go.Figure()

# 处理单位
fig.add_trace(go.Scatter(
    x=df['year'],
    y=df['california'],
    name='加州（实际）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'
))

# 合成控制
fig.add_trace(go.Scatter(
    x=df['year'],
    y=synthetic,
    name='合成加州（反事实）',
    line=dict(color=COLORS['synthetic'], width=3, dash='dash'),
    mode='lines+markers'
))

# 处理时点
fig.add_vline(x=T0, line_dash="dash", line_color="gray", 
              annotation_text="政策实施", annotation_position="top")

# 添加处理后的阴影区域
fig.add_vrect(
    x0=T0, x1=df['year'].max(),
    fillcolor="lightgray", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="后处理期", annotation_position="top left"
)

fig.update_layout(
    title='合成控制效果展示',
    xaxis_title='年份',
    yaxis_title='香烟消费量（包/人/年）',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n观察：")
print("  1. 前处理期（1988年前）：红线和蓝线几乎重合")
print("  2. 后处理期（1988年后）：红线低于蓝线 - 政策有效！")
print("  3. 两条线的差距就是处理效应")

---

# Part 3: 估计方法的细节

## 3.1 预测因子的选择

在实际应用中，我们通常不只匹配结果变量，还要匹配 **协变量**。

### 什么是预测因子？

预测因子（Predictors）是影响结果变量的协变量，例如：

**评估禁烟政策时的预测因子**：
- 人均收入
- 受教育程度
- 年龄结构
- 啤酒消费量（与吸烟相关的生活方式）
- 历史吸烟率（如 1980, 1985 年的值）

### 为什么要匹配协变量？

**直觉**：如果两个地区在所有重要特征上都相似，那么它们的结果变量趋势也应该相似。

**数学**：在某些假设下，匹配协变量可以保证合成控制是处理单位反事实的无偏估计。

---

## 3.2 V 矩阵的选择

回顾优化目标：

$$
\min_W (X_1 - X_0 W)^T V (X_1 - X_0 W)
$$

$V$ 是对角矩阵，体现不同预测因子的重要性：

$$
V = \text{diag}(v_1, v_2, ..., v_k)
$$

- $v_i$ 大 → 第 $i$ 个预测因子重要 → 优化时更关注匹配这个变量
- $v_i = 0$ → 忽略第 $i$ 个预测因子

### 如何选择 V？

**方法 1：等权重**
$$
V = I \quad \text{(单位矩阵)}
$$

**方法 2：交叉验证**

选择 $V$，使得合成控制在前处理期的拟合最好：

$$
V^* = \arg\min_V \sum_{t=1}^{T_0} \left( Y_{1t} - \sum_{j=2}^{J+1} w_j^*(V) Y_{jt} \right)^2
$$

这是一个嵌套优化问题（外层优化 $V$，内层优化 $W$）。

---

## 3.3 稀疏性：为什么很多权重为 0？

你可能注意到，估计出的权重往往很稀疏（很多 $w_j = 0$）。

**原因**：
1. **非负约束** + **和为 1 约束** → 自然倾向于稀疏解
2. **过拟合防止**：只用少数几个相似的供体，比用所有供体更稳健

**好处**：
- 解释性更强（"合成加州主要由纽约和佛州组成"）
- 避免过拟合

**类比**：LASSO 回归（L1 正则化）也会产生稀疏解。

---

In [ ]:
# ===================================
# TODO 1: 实现带协变量的合成控制
# ===================================

# 提示：
# 1. 创建 SyntheticControlWithCovariates 类
# 2. fit 方法需要同时接受 covariates_treated 和 covariates_donors 参数
# 3. 优化目标应该包含两部分：
#    - (1 - alpha) * 结果变量匹配误差
#    - alpha * 协变量匹配误差
# 4. 记得标准化数据以避免量纲问题

# 你的代码：


# 测试代码（取消注释后运行）
# city_features_dict = {
#     '纽约': [1.2, 0.8, 0.9],
#     '德州': [0.9, 1.0, 0.7],
#     '佛州': [0.8, 0.9, 0.6],
#     '伊利诺伊': [1.0, 0.7, 0.8],
#     '宾州': [0.7, 0.8, 0.7],
#     '俄亥俄': [0.6, 0.7, 0.6]
# }
# california_features = [1.5, 1.1, 1.0]
# covariates_donors_mat = np.array([city_features_dict[state] for state in state_names]).T
# covariates_treated_vec = np.array(california_features)
# 
# sc_cov = SyntheticControlWithCovariates(T0_index)
# sc_cov.fit(treated, donors, 
#            covariates_treated=covariates_treated_vec,
#            covariates_donors=covariates_donors_mat,
#            alpha=0.3)
# print(f"标准SCM效应: {sc.get_effect():.2f}")
# print(f"带协变量SCM效应: {sc_cov.get_effect():.2f}")

<details>
<summary>💡 点击查看参考答案</summary>

```python
class SyntheticControlWithCovariates:
    """
    带协变量匹配的合成控制（Augmented Synthetic Control）
    
    扩展标准合成控制,同时匹配结果变量和协变量
    """
    
    def __init__(self, treatment_period):
        self.treatment_period = treatment_period
        self.weights = None
        self.synthetic_control = None
        self.treatment_effect = None
        
    def fit(self, treated, donors, covariates_treated=None, covariates_donors=None, alpha=0.5):
        """
        估计合成控制权重
        
        参数:
        ------
        treated : array-like, shape (T,)
            处理单位的时间序列
        donors : array-like, shape (T, J)
            供体池的时间序列矩阵
        covariates_treated : array-like, shape (K,), optional
            处理单位的协变量向量
        covariates_donors : array-like, shape (K, J), optional
            供体池的协变量矩阵
        alpha : float, default=0.5
            协变量匹配的权重 (0-1)
            alpha=0: 只匹配结果变量
            alpha=1: 只匹配协变量
        """
        treated = np.array(treated)
        donors = np.array(donors)
        
        # 提取前处理期数据
        treated_pre = treated[:self.treatment_period]
        donors_pre = donors[:self.treatment_period, :]
        
        # 标准化（重要！）
        treated_pre_scaled = (treated_pre - np.mean(treated_pre)) / np.std(treated_pre)
        donors_pre_scaled = (donors_pre - np.mean(donors_pre, axis=0)) / np.std(donors_pre, axis=0)
        
        # 优化目标
        def objective(w):
            # Part 1: 结果变量匹配
            synthetic_pre = donors_pre_scaled @ w
            outcome_loss = np.sum((treated_pre_scaled - synthetic_pre) ** 2)
            
            # Part 2: 协变量匹配（如果提供）
            if covariates_treated is not None and covariates_donors is not None:
                cov_treated = np.array(covariates_treated)
                cov_donors = np.array(covariates_donors)
                
                # 标准化协变量
                cov_treated_scaled = (cov_treated - np.mean(cov_treated)) / (np.std(cov_treated) + 1e-8)
                cov_donors_scaled = (cov_donors - np.mean(cov_donors, axis=0)) / (np.std(cov_donors, axis=0) + 1e-8)
                
                synthetic_cov = cov_donors_scaled @ w
                covariate_loss = np.sum((cov_treated_scaled - synthetic_cov) ** 2)
                
                # 加权组合
                total_loss = (1 - alpha) * outcome_loss + alpha * covariate_loss
            else:
                total_loss = outcome_loss
            
            return total_loss
        
        # 约束条件
        constraints = (
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # 权重和为 1
        )
        
        # 边界：权重非负
        bounds = [(0, 1) for _ in range(donors.shape[1])]
        
        # 初始值：等权重
        w0 = np.ones(donors.shape[1]) / donors.shape[1]
        
        # 求解
        from scipy.optimize import minimize
        result = minimize(
            objective,
            w0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'ftol': 1e-9, 'maxiter': 1000}
        )
        
        if not result.success:
            print("优化未收敛")
        
        self.weights = result.x
        
        # 生成合成控制
        self.synthetic_control = donors @ self.weights
        
        # 计算处理效应
        self.treatment_effect = treated[self.treatment_period:] - self.synthetic_control[self.treatment_period:]
        
        return self
    
    def get_weights(self):
        """返回权重"""
        return self.weights
    
    def get_effect(self):
        """返回平均处理效应"""
        return np.mean(self.treatment_effect)
    
    def predict(self):
        """返回合成控制的预测值"""
        return self.synthetic_control

print("带协变量的合成控制实现完成")

# 应用示例
city_features_dict = {
    '纽约': [1.2, 0.8, 0.9],    # GDP指数, 人口指数, 产业结构
    '德州': [0.9, 1.0, 0.7],
    '佛州': [0.8, 0.9, 0.6],
    '伊利诺伊': [1.0, 0.7, 0.8],
    '宾州': [0.7, 0.8, 0.7],
    '俄亥俄': [0.6, 0.7, 0.6]
}

california_features = [1.5, 1.1, 1.0]

# 构造协变量矩阵
covariates_donors_mat = np.array([city_features_dict[state] for state in state_names]).T  # shape (K, J)
covariates_treated_vec = np.array(california_features)  # shape (K,)

# 拟合带协变量的模型
sc_cov = SyntheticControlWithCovariates(T0_index)
sc_cov.fit(treated, donors, 
           covariates_treated=covariates_treated_vec,
           covariates_donors=covariates_donors_mat,
           alpha=0.3)

print("\n" + "=" * 60)
print("权重对比：标准 SCM vs 带协变量 SCM")
print("=" * 60)
print(f"{'州':<12s} {'标准SCM':>12s} {'带协变量SCM':>15s}")
print("-" * 60)
for i, state in enumerate(state_names):
    print(f"{state:<12s} {weights[i]:>12.4f} {sc_cov.get_weights()[i]:>15.4f}")
print("=" * 60)
print(f"\n标准SCM效应: {sc.get_effect():.2f}")
print(f"带协变量SCM效应: {sc_cov.get_effect():.2f}")
print(f"差异: {abs(sc.get_effect() - sc_cov.get_effect()):.2f}")
print("\n解读:")
print("  • 带协变量的SCM不仅匹配历史趋势,还匹配城市特征")
print("  • 权重分布可能更倾向于特征相似的州")
print("  • 当协变量与结果高度相关时,这种方法通常更准确")
```

</details>

In [ ]:
# 实现 Placebo Tests
def placebo_test(treated_data, donors_data, treatment_period, donor_names):
    """
    执行 Placebo Tests
    
    参数:
    ------
    treated_data : array, shape (T,)
        真实处理单位数据
    donors_data : array, shape (T, J)
        供体池数据
    treatment_period : int
        处理时点索引
    donor_names : list
        供体名称列表
    
    返回:
    ------
    results : dict
        包含所有单位的效应、RMSPE 等信息
    """
    n_donors = donors_data.shape[1]
    n_periods = len(treated_data)
    
    # 存储结果
    effects = {}  # 平均处理效应
    gaps = {}     # 每个时期的 gap
    rmspe_pre = {}   # 前处理期 RMSPE
    rmspe_post = {}  # 后处理期 RMSPE
    
    # 1. 真实处理单位
    sc_real = SyntheticControl(treatment_period)
    sc_real.fit(treated_data, donors_data)
    
    synthetic_real = sc_real.predict()
    gap_real = treated_data - synthetic_real
    
    effects['加州（真实）'] = sc_real.get_effect()
    gaps['加州（真实）'] = gap_real
    rmspe_pre['加州（真实）'] = np.sqrt(np.mean(gap_real[:treatment_period]**2))
    rmspe_post['加州（真实）'] = np.sqrt(np.mean(gap_real[treatment_period:]**2))
    
    # 2. 对每个供体进行 Placebo 检验
    for i in range(n_donors):
        # 假装第 i 个供体是处理单位
        placebo_treated = donors_data[:, i]
        
        # 其他供体作为供体池
        placebo_donors = np.delete(donors_data, i, axis=1)
        
        # 估计合成控制
        sc_placebo = SyntheticControl(treatment_period)
        sc_placebo.fit(placebo_treated, placebo_donors)
        
        synthetic_placebo = sc_placebo.predict()
        gap_placebo = placebo_treated - synthetic_placebo
        
        name = f"{donor_names[i]}（Placebo）"
        effects[name] = sc_placebo.get_effect()
        gaps[name] = gap_placebo
        rmspe_pre[name] = np.sqrt(np.mean(gap_placebo[:treatment_period]**2))
        rmspe_post[name] = np.sqrt(np.mean(gap_placebo[treatment_period:]**2))
    
    return {
        'effects': effects,
        'gaps': gaps,
        'rmspe_pre': rmspe_pre,
        'rmspe_post': rmspe_post
    }

# 执行 Placebo Tests
placebo_results = placebo_test(treated, donors, T0_index, state_names)

print("Placebo Tests 完成")
print("\n各单位的平均处理效应:")
for name, effect in placebo_results['effects'].items():
    print(f"  {name}: {effect:.2f}")

In [ ]:
# 可视化：Placebo Tests - Gap 图
fig = go.Figure()

years = df['year'].values

# Placebo 单位（灰色细线）
for name, gap in placebo_results['gaps'].items():
    if 'Placebo' in name:
        fig.add_trace(go.Scatter(
            x=years,
            y=gap,
            name=name,
            line=dict(color=COLORS['donors'], width=1),
            opacity=0.3,
            showlegend=False
        ))

# 真实处理单位（粗红线）
fig.add_trace(go.Scatter(
    x=years,
    y=placebo_results['gaps']['加州（真实）'],
    name='加州（真实）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'
))

# 零线
fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

# 处理时点
fig.add_vline(x=T0, line_dash="dash", line_color="gray", 
              annotation_text="政策实施", annotation_position="top")

fig.update_layout(
    title='Placebo Tests: Gap 图',
    xaxis_title='年份',
    yaxis_title='Gap (实际 - 合成)',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n解读：")
print("  • 粗红线：真实处理单位的 Gap")
print("  • 细灰线：Placebo 单位的 Gap（如果它们被处理）")
print("  • 如果红线在后处理期远离灰线区域 - 效应显著")

In [ ]:
# 计算 p 值
real_effect = abs(placebo_results['effects']['加州（真实）'])
all_effects = [abs(v) for v in placebo_results['effects'].values()]

p_value = np.mean([e >= real_effect for e in all_effects])

print(f"\n统计推断结果：")
print(f"  真实效应: {placebo_results['effects']['加州（真实）']:.2f}")
print(f"  p 值: {p_value:.3f}")
print(f"  结论: {'显著' if p_value < 0.05 else '不显著'}")

# 效应分布
fig = go.Figure()

effects_list = list(placebo_results['effects'].values())
colors = ['red' if i == 0 else 'lightgray' for i in range(len(effects_list))]

fig.add_trace(go.Histogram(
    x=effects_list,
    nbinsx=15,
    marker_color=COLORS['synthetic'],
    opacity=0.7,
    name='Placebo 分布'
))

# 真实效应
fig.add_vline(
    x=placebo_results['effects']['加州（真实）'],
    line_color=COLORS['treated'],
    line_width=3,
    annotation_text=f"真实效应\np={p_value:.3f}",
    annotation_position="top right"
)

fig.update_layout(
    title='处理效应分布（含 Placebo）',
    xaxis_title='平均处理效应',
    yaxis_title='频数',
    template='plotly_white',
    height=400
)

fig.show()

In [ ]:
# Pre/Post RMSPE 比值检验
# 直觉：如果处理真的有效，那么后处理期的 RMSPE 应该显著增大

rmspe_ratios = {}
for name in placebo_results['rmspe_pre'].keys():
    ratio = placebo_results['rmspe_post'][name] / placebo_results['rmspe_pre'][name]
    rmspe_ratios[name] = ratio

# 可视化
names = list(rmspe_ratios.keys())
ratios = list(rmspe_ratios.values())
colors_bar = [COLORS['treated'] if '真实' in name else COLORS['placebo'] for name in names]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=names,
    y=ratios,
    marker_color=colors_bar,
    text=[f"{r:.2f}" for r in ratios],
    textposition='auto'
))

fig.update_layout(
    title='RMSPE 比值检验 (Post/Pre)',
    xaxis_title='单位',
    yaxis_title='RMSPE(后) / RMSPE(前)',
    template='plotly_white',
    height=400,
    xaxis_tickangle=-45
)

fig.show()

print("\n解读：")
print("  • 如果真实单位的比值远大于 Placebo 单位 - 效应显著")
print(f"  • 真实单位比值: {rmspe_ratios['加州（真实）']:.2f}")
print(f"  • Placebo 单位平均比值: {np.mean([v for k, v in rmspe_ratios.items() if 'Placebo' in k]):.2f}")

In [ ]:
# ===================================
# TODO 2: 对比 DID 和合成控制的估计结果
# ===================================

# 提示：
# 1. 将数据转换为长格式（panel data）
# 2. 使用 statsmodels.formula.api.ols 进行 DID 回归
# 3. 回归模型：consumption ~ treat + post + treat_post
# 4. 比较 DID 系数和合成控制的 ATT
# 5. 可视化两种方法的反事实

# 你的代码：


# 测试代码（取消注释后运行）
# import statsmodels.formula.api as smf
# 
# # 准备 DID 数据
# data_long = []
# for idx, year in enumerate(df['year']):
#     # 加州（处理组）
#     data_long.append({
#         'year': year,
#         'state': '加州',
#         'consumption': df.loc[idx, 'california'],
#         'treat': 1,
#         'post': 1 if year >= T0 else 0
#     })
#     # 其他州（对照组）
#     for state in state_names:
#         data_long.append({
#             'year': year,
#             'state': state,
#             'consumption': df.loc[idx, state],
#             'treat': 0,
#             'post': 1 if year >= T0 else 0
#         })
# 
# df_did = pd.DataFrame(data_long)
# df_did['treat_post'] = df_did['treat'] * df_did['post']
# 
# # DID 回归
# model_did = smf.ols('consumption ~ treat + post + treat_post', data=df_did).fit()
# did_coef = model_did.params['treat_post']
# 
# print(f"DID 估计: {did_coef:.2f}")
# print(f"SCM 估计: {sc.get_effect():.2f}")
# print(f"差异: {abs(did_coef - sc.get_effect()):.2f}")

<details>
<summary>💡 点击查看参考答案</summary>

```python
import statsmodels.formula.api as smf

# 准备 DID 数据
# 将数据转换为长格式 (panel data)
data_long = []
for idx, year in enumerate(df['year']):
    # 加州（处理组）
    data_long.append({
        'year': year,
        'state': '加州',
        'consumption': df.loc[idx, 'california'],
        'treat': 1,
        'post': 1 if year >= T0 else 0
    })
    
    # 其他州（对照组）
    for state in state_names:
        data_long.append({
            'year': year,
            'state': state,
            'consumption': df.loc[idx, state],
            'treat': 0,
            'post': 1 if year >= T0 else 0
        })

df_did = pd.DataFrame(data_long)
df_did['treat_post'] = df_did['treat'] * df_did['post']

# DID 回归
model_did = smf.ols('consumption ~ treat + post + treat_post', data=df_did).fit()

print("=" * 70)
print("方法对比：DID vs 合成控制")
print("=" * 70)

print("\n[1. DID 回归结果]")
print(model_did.summary().tables[1])

did_coef = model_did.params['treat_post']
did_se = model_did.bse['treat_post']
did_pval = model_did.pvalues['treat_post']

print(f"\nDID 估计量: {did_coef:.2f}")
print(f"标准误: {did_se:.2f}")
print(f"p-value: {did_pval:.4f}")
print(f"95% CI: [{did_coef - 1.96*did_se:.2f}, {did_coef + 1.96*did_se:.2f}]")

print(f"\n[2. 合成控制结果]")
sc_effect = sc.get_effect()
print(f"合成控制 ATT: {sc_effect:.2f}")

print(f"\n[3. 结果对比]")
print(f"DID 估计: {did_coef:.2f}")
print(f"SCM 估计: {sc_effect:.2f}")
print(f"差异: {abs(did_coef - sc_effect):.2f}")
print(f"相对差异: {abs(did_coef - sc_effect) / abs(sc_effect) * 100:.1f}%")

# 解释差异
print("\n[4. 为什么结果不同？]")
print("\n  DID 方法:")
print("    • 对照组 = 所有其他州的简单平均（等权重）")
print("    • 假设: 平行趋势（所有州的趋势相同）")
print("    • 问题: 德州、纽约、阿拉斯加差异巨大，简单平均不合理")

print("\n  合成控制方法:")
print("    • 对照组 = 优化权重的加权平均")
print("    • 权重分布:", end="")
for i, state in enumerate(state_names):
    if weights[i] > 0.01:
        print(f" {state}({weights[i]:.2f})", end="")
print()
print("    • 更灵活，不依赖平行趋势假设")

print("\n  结论:")
if abs(did_coef - sc_effect) / abs(sc_effect) < 0.2:
    print("    • 两种方法结果接近，说明平行趋势假设基本成立")
else:
    print("    • 两种方法结果差异较大，平行趋势假设可能不成立")
    print("    • 此时应优先信任合成控制的结果")

# 可视化对比
fig = go.Figure()

# 加州实际
fig.add_trace(go.Scatter(
    x=df['year'], y=df['california'],
    name='加州（实际）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'))

# 合成控制反事实
fig.add_trace(go.Scatter(
    x=df['year'], y=synthetic,
    name='合成控制（反事实）',
    line=dict(color=COLORS['synthetic'], width=2, dash='dash'),
))

# DID 反事实
# DID反事实 = 处理组政策前均值 + (对照组政策后均值 - 对照组政策前均值)
treat_pre_mean = df['california'][:T0_index].mean()
control_pre_mean = df[state_names].iloc[:T0_index].mean().mean()
control_post_mean = df[state_names].iloc[T0_index:].mean().mean()

did_counterfactual = np.full(len(df), np.nan)
did_counterfactual[:T0_index] = df['california'][:T0_index].values
did_counterfactual[T0_index:] = treat_pre_mean + (control_post_mean - control_pre_mean)

fig.add_trace(go.Scatter(
    x=df['year'], y=did_counterfactual,
    name='DID（反事实）',
    line=dict(color=COLORS['gap'], width=2, dash='dot'),
))

# 处理时点
fig.add_vline(x=T0, line_dash="dash", line_color="gray",
              annotation_text="政策实施", annotation_position="top")

fig.update_layout(
    title='DID vs 合成控制：反事实对比',
    xaxis_title='年份',
    yaxis_title='香烟消费量（包/人/年）',
    template='plotly_white',
    height=500,
    hovermode='x unified')

fig.show()

print("=" * 70)
```

</details>

In [ ]:
# 生成业务案例数据
np.random.seed(123)

# 时间跨度
months = pd.date_range('2022-01', '2024-07', freq='M')
n_months = len(months)
treatment_month = 18  # 2023年7月（索引从0开始）

# 城市
cities = ['上海', '北京', '广州', '深圳', '成都', '杭州', '南京', '武汉', '西安']
n_donors = len(cities) - 1

# 城市特征（协变量）
city_features = pd.DataFrame({
    '城市': cities,
    '人口': [2500, 2200, 1900, 1800, 2100, 1200, 900, 1400, 1300],  # 万人
    'GDP': [4300, 4200, 2900, 3200, 2100, 1900, 1700, 1900, 1100],  # 亿元
    '互联网渗透率': [85, 82, 78, 80, 75, 83, 76, 72, 70],  # %
    '餐饮业规模': [380, 350, 280, 300, 200, 180, 150, 160, 120]  # 亿元
})

# 生成 GMV 数据
gmv_data = {}

for i, city in enumerate(cities):
    # 基线 GMV（与城市规模相关）
    base = city_features.loc[i, 'GDP'] * 0.5 + np.random.normal(0, 50)
    
    # 增长趋势（与互联网渗透率相关）
    trend = city_features.loc[i, '互联网渗透率'] * 0.3
    
    # 季节性
    seasonality = 50 * np.sin(np.arange(n_months) * 2 * np.pi / 12)
    
    # 噪音
    noise = np.random.normal(0, 30, n_months)
    
    # 组合
    gmv = base + trend * np.arange(n_months) + seasonality + noise
    
    # 上海的处理效应
    if city == '上海':
        treatment_effect_city = 200  # 平均 +200 万元/月
        # 效应逐渐增强（学习曲线）
        effect_curve = treatment_effect_city * (1 - np.exp(-0.3 * np.arange(n_months - treatment_month)))
        gmv[treatment_month:] += effect_curve
    
    gmv_data[city] = gmv

# 创建 DataFrame
gmv_df = pd.DataFrame(gmv_data)
gmv_df['month'] = months
gmv_df = gmv_df[['month'] + cities]  # 调整列顺序

print("业务案例数据生成完成")
print(f"\n城市特征:")
print(city_features)
print(f"\nGMV 数据 (前5月):")
print(gmv_df.head())

---

# Part 6: 业务案例

## 案例 1：新城市上线效果评估

### 背景

你的公司是一家外卖平台，2023 年 7 月在上海上线。你需要评估上线的因果效应。

### 数据

- **处理单位**：上海
- **供体池**：北京、广州、深圳、成都、杭州、南京、武汉、西安
- **时间跨度**：2022 年 1 月 - 2024 年 6 月（共 30 个月）
- **处理时点**：2023 年 7 月（第 19 个月）
- **结果变量**：月度 GMV（万元）
- **协变量**：
  - 人口（万人）
  - GDP（亿元）
  - 互联网渗透率（%）
  - 餐饮业规模（亿元）

---

In [ ]:
# ===================================
# TODO 3: 评估上海上线的因果效应
# ===================================

# 任务：完成一个完整的因果效应评估流程
# 
# Step 1: 数据准备
#   - 提取上海的 GMV 数据作为 treated
#   - 提取其他城市的 GMV 数据作为 donors
# 
# Step 2: 估计合成控制
#   - 使用 SyntheticControl 类拟合模型
#   - 获取权重并分析哪些城市权重最高
# 
# Step 3: 拟合质量检查
#   - 计算前处理期和后处理期的 RMSPE
#   - 检查拟合是否良好
# 
# Step 4: 处理效应估计
#   - 计算 ATT（绝对值和相对值）
#   - 分析逐月效应
# 
# Step 5: 可视化
#   - 绘制实际 vs 合成控制对比图
# 
# Step 6: Placebo Tests
#   - 执行 placebo_test 函数
#   - 计算 p 值判断显著性
#   - 绘制 Gap 图
# 
# Step 7: 业务建议
#   - 根据分析结果给出业务建议

# 你的代码：

<details>
<summary>💡 点击查看参考答案</summary>

```python
print("=" * 70)
print("完整的因果效应评估流程：上海功能上线案例")
print("=" * 70)

# Step 1: 数据准备
treated_shanghai = gmv_df['上海'].values
donors_shanghai = gmv_df[cities[1:]].values

# Step 2: 估计合成控制
sc_shanghai = SyntheticControl(treatment_month)
sc_shanghai.fit(treated_shanghai, donors_shanghai)

synthetic_shanghai = sc_shanghai.predict()
weights_shanghai = sc_shanghai.get_weights()

print("\n[Step 1: 合成控制权重]")
print("-" * 70)
weight_df_sh = pd.DataFrame({
    '城市': cities[1:],
    '权重': weights_shanghai
}).sort_values('权重', ascending=False)

print(weight_df_sh.to_string(index=False))
print(f"\n有意义的权重（>1%）: {(weight_df_sh['权重'] > 0.01).sum()} 个城市")

# Step 3: 拟合质量检查
pre_period_mask = np.arange(treatment_month)
post_period_mask = np.arange(treatment_month, len(treated_shanghai))

gap = treated_shanghai - synthetic_shanghai

rmspe_pre = np.sqrt(np.mean(gap[pre_period_mask]**2))
rmspe_post = np.sqrt(np.mean(gap[post_period_mask]**2))

print(f"\n[Step 2: 拟合质量]")
print("-" * 70)
print(f"前处理期 RMSPE: {rmspe_pre:.2f}")
print(f"后处理期 RMSPE: {rmspe_post:.2f}")
print(f"RMSPE 比值: {rmspe_post / rmspe_pre:.2f}")

# Step 4: 处理效应估计
att = sc_shanghai.get_effect()
att_pct = att / np.mean(synthetic_shanghai[post_period_mask]) * 100

print(f"\n[Step 3: 处理效应]")
print("-" * 70)
print(f"ATT（绝对）: {att:.2f} 万元/月")
print(f"ATT（相对）: {att_pct:.2f}%")

# 逐月效应
print(f"\n逐月效应（后处理期）:")
for i, month_idx in enumerate(post_period_mask):
    month = gmv_df['month'].iloc[month_idx]
    effect = gap[month_idx]
    print(f"  {month.strftime('%Y-%m')}: {effect:+.2f}")

# Step 5: 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=gmv_df['month'], y=treated_shanghai,
    name='上海（实际）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'))

fig.add_trace(go.Scatter(
    x=gmv_df['month'], y=synthetic_shanghai,
    name='合成上海（反事实）',
    line=dict(color=COLORS['synthetic'], width=3, dash='dash'),
    mode='lines+markers'))

treatment_date = gmv_df['month'].iloc[treatment_month]
fig.add_vline(x=treatment_date, line_dash="dash", line_color="gray",
              annotation_text="上海上线", annotation_position="top")

fig.add_vrect(
    x0=treatment_date, x1=gmv_df['month'].iloc[-1],
    fillcolor="lightgray", opacity=0.2,
    layer="below", line_width=0,
)

fig.update_layout(
    title='上海上线的因果效应：合成控制分析',
    xaxis_title='月份',
    yaxis_title='GMV（万元）',
    template='plotly_white',
    height=500,
    hovermode='x unified')

fig.show()

# Step 6: Placebo Tests
print(f"\n[Step 4: Placebo Tests]")
print("-" * 70)

placebo_results_sh = placebo_test(
    treated_shanghai, donors_shanghai, treatment_month, cities[1:])

# 计算 p 值
real_effect_sh = abs(att)
all_effects_sh = [abs(v) for v in placebo_results_sh['effects'].values()]
p_value_sh = np.mean([e >= real_effect_sh for e in all_effects_sh])

print(f"真实效应: {att:.2f}")
print(f"p-value: {p_value_sh:.3f}")
print(f"结论: {'显著' if p_value_sh < 0.05 else '不显著'}")

# Gap 图
fig2 = go.Figure()

# Placebo gaps
for name, gap_series in placebo_results_sh['gaps'].items():
    if 'Placebo' in name:
        fig2.add_trace(go.Scatter(
            x=gmv_df['month'], y=gap_series,
            name=name,
            line=dict(color=COLORS['donors'], width=1),
            opacity=0.3,
            showlegend=False
        ))

# Real gap
fig2.add_trace(go.Scatter(
    x=gmv_df['month'], y=placebo_results_sh['gaps']['上海（真实）'],
    name='上海（真实）',
    line=dict(color=COLORS['treated'], width=3),
))

fig2.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
fig2.add_vline(x=treatment_date, line_dash="dash", line_color="gray")

fig2.update_layout(
    title='Placebo Tests: Gap 图',
    xaxis_title='月份',
    yaxis_title='Gap（实际 - 合成）',
    template='plotly_white',
    height=500)

fig2.show()

# Step 7: 业务建议
print(f"\n[Step 5: 业务建议]")
print("=" * 70)

if p_value_sh < 0.05 and att > 0:
    print(f"结论：上海上线效果显著且为正")
    print(f"\n证据：")
    print(f"  1. ATT = {att:.2f} 万元/月（相对提升 {att_pct:.2f}%）")
    print(f"  2. Placebo test p-value = {p_value_sh:.3f} < 0.05（统计显著）")
    print(f"  3. 前处理期拟合良好（RMSPE = {rmspe_pre:.2f}）")
    print(f"\n建议：")
    print(f"  • 继续在上海运营，巩固市场地位")
    print(f"  • 可考虑将上海的成功经验复制到其他一线城市")
    print(f"  • 预计推广到{len(cities)-1}个类似城市，年增收:")
    print(f"    {att * 12 * (len(cities)-1):.0f} 万元")
elif p_value_sh >= 0.05:
    print(f"结论：上海上线效果不显著")
    print(f"\n可能原因：")
    print(f"  1. 样本量不足（观测期过短）")
    print(f"  2. 效应确实较小，被噪音淹没")
    print(f"  3. 需要更长时间才能看到效应")
    print(f"\n建议：")
    print(f"  • 延长观测期至6个月以上")
    print(f"  • 检查上海的运营数据，寻找改进点")
    print(f"  • 暂缓向其他城市扩张")
else:
    print(f"结论：上海上线效果显著为负")
    print(f"\n需要紧急调查：")
    print(f"  • 定价策略是否合理？")
    print(f"  • 运营团队是否到位？")
    print(f"  • 本地竞争是否超预期？")

print("=" * 70)
```

</details>

---

## 案例 2：大客户流失的影响评估

### 背景

你是一家 SaaS 公司的数据分析师。2023 年 3 月，公司最大的客户 A（占总收入 15%）因价格原因流失。

老板问："A 客户流失后，我们的增长率下降了 5%，这 5% 都是 A 客户造成的吗？"

### 分析思路

1. **处理单位**：你的公司
2. **供体池**：其他类似的 SaaS 公司（没有大客户流失）
3. **结果变量**：月度增长率（MRR Month-over-Month Growth）
4. **处理时点**：2023 年 3 月

---

In [ ]:
# 生成大客户流失案例数据
np.random.seed(456)

months_churn = pd.date_range('2021-01', '2023-12', freq='M')
n_months_churn = len(months_churn)
churn_month = 26  # 2023年3月

# 公司
companies = ['你的公司', '竞品A', '竞品B', '竞品C', '竞品D', '竞品E']

# 生成增长率数据（%）
growth_data = {}

for company in companies:
    # 基线增长率
    base_growth = np.random.uniform(3, 5)
    
    # 趋势（SaaS 增长通常逐渐放缓）
    trend = -0.02 * np.arange(n_months_churn)
    
    # 噪音
    noise = np.random.normal(0, 0.5, n_months_churn)
    
    growth = base_growth + trend + noise
    
    # 你的公司：大客户流失导致增长率下降
    if company == '你的公司':
        churn_impact = -2.5  # 增长率下降 2.5%
        growth[churn_month:] += churn_impact
    
    growth_data[company] = growth

growth_df = pd.DataFrame(growth_data)
growth_df['month'] = months_churn
growth_df = growth_df[['month'] + companies]

print("大客户流失案例数据生成完成")
print(growth_df.head())

In [ ]:
# 应用合成控制
treated_churn = growth_df['你的公司'].values
donors_churn = growth_df[companies[1:]].values

sc_churn = SyntheticControl(churn_month)
sc_churn.fit(treated_churn, donors_churn)

synthetic_churn = sc_churn.predict()

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=growth_df['month'],
    y=growth_df['你的公司'],
    name='你的公司（实际）',
    line=dict(color=COLORS['treated'], width=3),
    mode='lines+markers'
))

fig.add_trace(go.Scatter(
    x=growth_df['month'],
    y=synthetic_churn,
    name='合成公司（反事实）',
    line=dict(color=COLORS['synthetic'], width=3, dash='dash'),
    mode='lines+markers'
))

# 客户流失时点
churn_date = months_churn[churn_month]
fig.add_vline(x=churn_date, line_dash="dash", line_color="gray",
              annotation_text="客户A流失", annotation_position="top")

fig.update_layout(
    title='大客户流失的因果效应',
    xaxis_title='月份',
    yaxis_title='MoM 增长率（%）',
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print(f"\n估计的客户流失影响: {sc_churn.get_effect():.2f}%")
print(f"\n业务解读：")
print(f"  • 客户 A 流失后，增长率下降了 {abs(sc_churn.get_effect()):.2f}%")
print(f"  • 这是纯粹的因果效应（排除了市场整体趋势）")

---

# Part 7: 练习与思考题

## 练习题

### 练习 1：实现预测因子匹配

扩展 `SyntheticControl` 类，支持协变量匹配。用案例 1 的数据（城市 GMV + 城市特征），重新估计上海上线的效应。

### 练习 2：时间 Placebo 检验

除了单位 Placebo（假装其他单位接受处理），还可以做时间 Placebo（假装处理发生在其他时间点）。实现时间 Placebo 检验。

### 练习 3：多个处理单位的合成控制

假设你的公司在 2023 年 7 月同时在上海和成都上线。如何用合成控制评估两个城市的平均效应？

---

## 思考题

### 思考 1：外推的风险

合成控制本质上是 **内插**（Interpolation）还是 **外推**（Extrapolation）？如果处理单位的特征在供体池的凸包（Convex Hull）之外，会有什么问题？

### 思考 2：稀疏性的代价

权重稀疏（很多 $w_j = 0$）有什么好处？有什么坏处？如果想要更平滑的权重，可以怎么做？

**提示**：考虑在优化目标中加入正则化项，如：
$$
\min_W \|X_1 - X_0 W\|^2 + \lambda \|W\|_2^2
$$

### 思考 3：因果识别的本质

合成控制的因果识别假设是什么？和潜在结果框架（Potential Outcomes）的关系是什么？

**核心假设**：
$$
Y_{1t}^N = \sum_{j=2}^{J+1} w_j Y_{jt} + \epsilon_t, \quad E[\epsilon_t] = 0
$$

这个假设强吗？什么情况下会违反？

---

## 延伸阅读

### 经典论文

1. **Abadie, A., & Gardeazabal, J. (2003).** "The Economic Costs of Conflict: A Case Study of the Basque Country." *American Economic Review*.
   - 合成控制的开创性论文

2. **Abadie, A., Diamond, A., & Hainmueller, J. (2010).** "Synthetic Control Methods for Comparative Case Studies." *Journal of the American Statistical Association*.
   - 方法论的系统阐述

3. **Abadie, A., Diamond, A., & Hainmueller, J. (2015).** "Comparative Politics and the Synthetic Control Method." *American Journal of Political Science*.
   - 应用指南 + 加州禁烟案例

### Python 工具包

- **SparseSC**: https://github.com/Microsoft/SparseSC
  - 微软开发的合成控制包，支持正则化、多个处理单位等

- **CausalImpact** (R 移植): https://github.com/jamalsenouci/causalimpact
  - 基于贝叶斯结构时间序列的因果推断（Google 的方法）

---

## 小结

通过本节课，我们学习了：

1. ✅ **合成控制的思想**：用供体池的加权组合构建反事实
2. ✅ **权重估计方法**：二次规划问题，优化前处理期拟合
3. ✅ **推断方法**：Placebo Tests、排列 p 值
4. ✅ **与 DID 的关系**：合成控制是 DID 的推广
5. ✅ **业务应用**：新城市上线、大客户流失等场景

### 核心要点

- 合成控制适用于 **单个处理单位** 的场景
- 关键是找到 **权重**，使得合成控制在前处理期尽可能拟合处理单位
- **Placebo Tests** 是推断显著性的标准方法
- 合成控制比 DID 更 **灵活**，但也需要更多供体单位

### 下节预告

下节课我们将学习 **断点回归设计（Regression Discontinuity Design, RDD）**，另一种经典的准实验方法。

---

**恭喜你完成了合成控制法的学习！🎉**